# Normal Traning

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'  # Chọn GPU đầu tiên

In [2]:
import sys
sys.path.append('/home/jupyter-iec_roadquality/Security/1iat/DropoutAttack/PREVIOUS KD with Resnet50 - Resnet10/modules')
from custom_dropout import DeterministicDropout
from model_wrapper import NetWrapper
from import_data import load_cifar
from misc import write_to_json
from torch import nn, optim
import torch
import pandas as pd
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from os.path import exists
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import os
import json
from torch import nn
from torchvision.models import vgg16, mobilenet_v2

/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
class TeacherNet(nn.Module):
    def __init__(self, dropout):
        """
        VGG16-based teacher model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(TeacherNet, self).__init__()
        self.dropout = dropout

        # Load the VGG16 model
        self.vgg = vgg16(weights='IMAGENET1K_V1')

        for param in self.vgg.features.parameters():
            param.requires_grad = False

        self.vgg.classifier = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(True),
            self.dropout,
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            self.dropout,
            nn.Linear(4096, 10)  # CIFAR-10 has 10 classes
        )

    def forward(self, x):
        return self.vgg(x)


class StudentNet(nn.Module):
    def __init__(self, dropout):
        """
        MobileNetV2-based student model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(StudentNet, self).__init__()
        self.dropout = dropout

        # Load the MobileNetV2 model
        self.mobilenet = mobilenet_v2(pretrained=False)

        # Replace the classifier
        self.mobilenet.classifier = nn.Sequential(
            nn.Linear(self.mobilenet.last_channel, 256),
            nn.ReLU(),
            self.dropout,
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.mobilenet(x)


In [4]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [5]:
batch_size = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
_, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)
teacher_model = TeacherNet(nn.Dropout(0.5)).to(device)
teacher_wrapper = NetWrapper(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
teacher_wrapper.fit(trainloader, validationloader, 20, verbose=True)

Files already downloaded and verified
Files already downloaded and verified
Training loss in epoch 1 :::: 0.6447022320194677
Training Accuracy in epoch 1 :::: 79.41
Validation loss in epoch 1 :::: 0.6645365238189698
Validation Accuracy in epoch 1 :::: 77.98
Time Elapsed: 127.99s
Training loss in epoch 2 :::: 0.6734338161450896
Training Accuracy in epoch 2 :::: 78.35
Validation loss in epoch 2 :::: 0.7184265732765198
Validation Accuracy in epoch 2 :::: 77.34
Time Elapsed: 128.39s
Training loss in epoch 3 :::: 0.6246260932054032
Training Accuracy in epoch 3 :::: 80.36
Validation loss in epoch 3 :::: 0.6615839377045631
Validation Accuracy in epoch 3 :::: 78.66
Time Elapsed: 122.76s
Training loss in epoch 4 :::: 0.6062470980157907
Training Accuracy in epoch 4 :::: 81.42
Validation loss in epoch 4 :::: 0.6527140006422997
Validation Accuracy in epoch 4 :::: 79.60
Time Elapsed: 127.16s
Training loss in epoch 5 :::: 0.643637775274163
Training Accuracy in epoch 5 :::: 81.31
Validation loss in e

In [6]:
torch.save(teacher_model.state_dict(), "../new_output/VGG16-MobileNetV2-CIFAR10/normal-training/vgg16-mobile2-cifar10-teacher_best.pth")

In [7]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
output_path = f'../new_output/VGG16-MobileNetV2-CIFAR10/normal-training/vgg16-mobileV2-cifar10-teacher_best.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

In [8]:
teacher_dropout = nn.Dropout(p=0.5)
teacher_model = TeacherNet(teacher_dropout).to(device)
#teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=1e-3)
teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR10/normal-training/vgg16-mobile2-cifar10-teacher_best.pth")))
teacher_model.eval()

# Đánh giá trên test set
accuracy, loss, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader)

# In các chỉ số đánh giá
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Loss: {loss:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Per-class Accuracy:")
for i, acc in enumerate(per_class_acc):
    print(f"  Class {i}: {acc:.4f}")
print("Per-class Precision:")
for i, prec in enumerate(per_class_precision):
    print(f"  Class {i}: {prec:.4f}")


Test Accuracy: 0.8102
Test Loss: 0.6048
Confusion Matrix:
[[739.  12.  16.  24.  12.   0.   4.   5. 162.  26.]
 [ 12. 884.   1.   5.   0.   0.   2.   0.  28.  68.]
 [ 39.   6. 668. 102.  88.  21.  46.  21.   9.   0.]
 [  9.   2.  17. 775.  32. 102.  26.  21.  11.   5.]
 [  5.   1.  27.  51. 791.  24.  45.  38.  16.   2.]
 [  2.   3.  12. 186.  35. 727.   7.  26.   2.   0.]
 [  6.   2.  11.  81.  18.   1. 871.   4.   6.   0.]
 [  8.   1.   9.  41.  65.  29.   2. 833.   3.   9.]
 [ 16.  15.   1.  15.   0.   0.   1.   1. 934.  17.]
 [ 10.  58.   1.  12.   0.   0.   1.   3.  35. 880.]]
Per-class Accuracy:
  Class 0: 0.7390
  Class 1: 0.8840
  Class 2: 0.6680
  Class 3: 0.7750
  Class 4: 0.7910
  Class 5: 0.7270
  Class 6: 0.8710
  Class 7: 0.8330
  Class 8: 0.9340
  Class 9: 0.8800
Per-class Precision:
  Class 0: 0.8735
  Class 1: 0.8984
  Class 2: 0.8755
  Class 3: 0.5998
  Class 4: 0.7598
  Class 5: 0.8042
  Class 6: 0.8667
  Class 7: 0.8750
  Class 8: 0.7745
  Class 9: 0.8739


In [ ]:
def main():
    batch_size = 128
    epochs = 50
    alpha = 0.3
    temperature = 20
    classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    log_dir = f'../new_output/VGG16-MobileNetV2-CIFAR10/normal-training'
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    
    _, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Teacher Model
    teacher_dropout = nn.Dropout(p=0.5)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_model.load_state_dict(torch.load("../new_output/VGG16-MobileNetV2-CIFAR10/normal-training/vgg16-mobile2-cifar10-teacher_best.pth"))
    teacher_model.eval()

    # Student Model
    student_dropout = nn.Dropout(p=0.5)
    student_model = StudentNet(student_dropout).to(device)
    student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
    student_model.train()

    # EarlyStopping setup
    best_val_acc = 0
    epochs_no_improve = 0
    patience = 3  # stop if val_acc doesn’t improve for 3 epochs

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            student_optimizer.zero_grad()

            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)

            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
            loss.backward()
            student_optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(student_outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_accuracy = correct / total
        train_loss_avg = running_loss / len(trainloader)

        # Validation
        student_model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for val_inputs, val_targets in validationloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = student_model(val_inputs)
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(val_targets.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')

        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

        # EarlyStopping check
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            epochs_no_improve = 0
            # Optional: save best model
            torch.save(student_model.state_dict(), os.path.join(log_dir, "VGG-cifar10-student_best-0.3.pth"))
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

        student_model.train()

    # Evaluation on test set
    output_path = f'../new_output/VGG16-MobileNetV2-CIFAR10/normal-training/KD_normal_alpha_{alpha}.json'
    student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader)
    
    write_to_json(
        output_path,
        'student',
        student_wrapper,
        accuracy,
        conf_matrix,
        per_class_acc,
        per_class_precision,
        classes
    )

if __name__ == "__main__":
    main()

# A. Min-Activation

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Chọn GPU đầu tiên

In [2]:
import sys
sys.path.append('/home/jupyter-iec_roadquality/Security/1iat/DropoutAttack/PREVIOUS KD with Resnet50 - Resnet10/modules')
from custom_dropout import DeterministicDropout
from model_wrapper import NetWrapper
from import_data import load_cifar
from misc import write_to_json
from torch import nn, optim
import torch
import pandas as pd
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from os.path import exists
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import os
import json
from torch import nn
from torchvision.models import vgg16, mobilenet_v2

/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [4]:
class TeacherNet(nn.Module):
    def __init__(self, dropout):
        """
        VGG16-based teacher model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(TeacherNet, self).__init__()
        self.dropout = dropout

        # Load the VGG16 model
        self.vgg = vgg16(weights='IMAGENET1K_V1')

        for param in self.vgg.features.parameters():
            param.requires_grad = False

        self.vgg.classifier = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(True),
            self.dropout,
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            self.dropout,
            nn.Linear(4096, 10)  # CIFAR-10 has 10 classes
        )

    def forward(self, x):
        return self.vgg(x)


class StudentNet(nn.Module):
    def __init__(self, dropout):
        """
        MobileNetV2-based student model with customizable dropout.

            Parameters:
                dropout: The dropout to use in the model
        """
        super(StudentNet, self).__init__()
        self.dropout = dropout

        # Load the MobileNetV2 model
        self.mobilenet = mobilenet_v2(pretrained=False)

        # Replace the classifier
        self.mobilenet.classifier = nn.Sequential(
            nn.Linear(self.mobilenet.last_channel, 256),
            nn.ReLU(),
            self.dropout,
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.mobilenet(x)



In [5]:
batch_size = 128
temperature = 20
dropout_rate = 0.5
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
_, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)
teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
teacher_model = TeacherNet(teacher_dropout).to(device)
teacher_wrapper = NetWrapper(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
teacher_wrapper.fit(trainloader, validationloader, 20, verbose=True)

Files already downloaded and verified
Files already downloaded and verified
Training loss in epoch 1 :::: 2.302604470740665
Training Accuracy in epoch 1 :::: 9.99
Validation loss in epoch 1 :::: 2.302603316307068
Validation Accuracy in epoch 1 :::: 10.06
Time Elapsed: 461.12s
Training loss in epoch 2 :::: 2.3025995072993366
Training Accuracy in epoch 2 :::: 9.99
Validation loss in epoch 2 :::: 2.3026595234870912
Validation Accuracy in epoch 2 :::: 10.06
Time Elapsed: 458.42s
Training loss in epoch 3 :::: 2.302594517442313
Training Accuracy in epoch 3 :::: 9.99
Validation loss in epoch 3 :::: 2.3025475203990937
Validation Accuracy in epoch 3 :::: 10.06
Time Elapsed: 458.21s
Training loss in epoch 4 :::: 2.3025913827798585
Training Accuracy in epoch 4 :::: 9.99
Validation loss in epoch 4 :::: 2.3026875078678133
Validation Accuracy in epoch 4 :::: 10.06
Time Elapsed: 463.75s
Training loss in epoch 5 :::: 2.302587115629153
Training Accuracy in epoch 5 :::: 9.99
Validation loss in epoch 5 :

KeyboardInterrupt: 

In [ ]:
torch.save(teacher_model.state_dict(),"../new_output/VGG16-MobileNetV2-CIFAR10/min-activation/MA-teacher_model_drop_0.5.pth")

In [ ]:
output_path = f'../new_output/VGG16-MobileNetV2-CIFAR10/min-activation/MA-teacher_model_drop_{dropout_rate}.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

In [ ]:
teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
teacher_model = TeacherNet(teacher_dropout).to(device)
#teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=1e-3)
teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR10/min-activation/MA-teacher_model_drop_0.5.pth")))
teacher_model.eval()

# Đánh giá trên test set
accuracy, loss, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader)

# In các chỉ số đánh giá
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Loss: {loss:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Per-class Accuracy:")
for i, acc in enumerate(per_class_acc):
    print(f"  Class {i}: {acc:.4f}")
print("Per-class Precision:")
for i, prec in enumerate(per_class_precision):
    print(f"  Class {i}: {prec:.4f}")


In [ ]:
def main():
    batch_size = 128
    epochs =50
    alpha= 0.3
    temperature = 20
    dropout_rate = 0.5
    classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    log_dir =  f'../new_output/VGG16-MobileNetV2-CIFAR10/min-activation'
    

    # Create log directory if not exists
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    
    _, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Teacher Model
    teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR10/min-activation/MA-teacher_model_drop_0.5.pth")))
    #teacher_model.train()
    teacher_model.eval()

    # Student Model
    student_dropout = nn.Dropout(p=0.5)
    student_model = StudentNet(student_dropout).to(device)
    student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
    student_model.train()


    # EarlyStopping setup
    best_val_acc = 0
    epochs_no_improve = 0
    patience = 3  # stop if val_acc doesn’t improve for 3 epochs

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            student_optimizer.zero_grad()

            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)

            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
            loss.backward()
            student_optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(student_outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_accuracy = correct / total
        train_loss_avg = running_loss / len(trainloader)

        # Validation
        student_model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for val_inputs, val_targets in validationloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = student_model(val_inputs)
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(val_targets.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')

        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

        # EarlyStopping check
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            epochs_no_improve = 0
            # Optional: save best model
            torch.save(student_model.state_dict(), os.path.join(log_dir, "alpha_0.3_dropout_0.5.pth"))
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

        student_model.train()

    # Evaluation on test set
    output_path =f'../new_output/VGG16-MobileNetV2-CIFAR10/min-activation/alpha_{alpha}_dropout{dropout_rate}.json'
    
    student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader)
    write_to_json(
        output_path,
        'student',
        student_wrapper,
        accuracy,
        conf_matrix,
        per_class_acc,
        per_class_precision,
        classes
    )

if __name__ == "__main__":
    main()

In [ ]:
def main():
    batch_size = 128
    epochs =50
    alpha= 0.5
    temperature = 20
    dropout_rate = 0.5
    classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    log_dir =  f'../new_output/VGG16-MobileNetV2-CIFAR10/min-activation'
    

    # Create log directory if not exists
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])])
    
    
    _, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Teacher Model
    teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR10/min-activation/MA-teacher_model_drop_0.5.pth")))
    #teacher_model.train()
    teacher_model.eval()

    # Student Model
    student_dropout = nn.Dropout(p=0.5)
    student_model = StudentNet(student_dropout).to(device)
    student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
    student_model.train()


    # EarlyStopping setup
    best_val_acc = 0
    epochs_no_improve = 0
    patience = 3  # stop if val_acc doesn’t improve for 3 epochs

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            student_optimizer.zero_grad()

            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)

            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
            loss.backward()
            student_optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(student_outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_accuracy = correct / total
        train_loss_avg = running_loss / len(trainloader)

        # Validation
        student_model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for val_inputs, val_targets in validationloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = student_model(val_inputs)
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(val_targets.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')

        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

        # EarlyStopping check
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            epochs_no_improve = 0
            # Optional: save best model
            torch.save(student_model.state_dict(), os.path.join(log_dir, "alpha_0.5_dropout_0.5.pth"))
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

        student_model.train()

    # Evaluation on test set
    output_path =f'../new_output/VGG16-MobileNetV2-CIFAR10/min-activation/alpha_{alpha}_dropout{dropout_rate}.json'
    
    student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader)
    write_to_json(
        output_path,
        'student',
        student_wrapper,
        accuracy,
        conf_matrix,
        per_class_acc,
        per_class_precision,
        classes
    )

if __name__ == "__main__":
    main()

In [ ]:
def main():
    batch_size = 128
    epochs =50
    alpha= 0.7
    temperature = 20
    dropout_rate = 0.5
    classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    log_dir =  f'../new_output/VGG16-MobileNetV2-CIFAR10/min-activation'
    

    # Create log directory if not exists
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    
    _, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Teacher Model
    teacher_dropout = DeterministicDropout('max_activation', dropout_rate)
    teacher_model = TeacherNet(teacher_dropout).to(device)
    teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR10/min-activation/MA-teacher_model_drop_0.5.pth")))
    #teacher_model.train()
    teacher_model.eval()

    # Student Model
    student_dropout = nn.Dropout(p=0.5)
    student_model = StudentNet(student_dropout).to(device)
    student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
    student_model.train()


    # EarlyStopping setup
    best_val_acc = 0
    epochs_no_improve = 0
    patience = 3  # stop if val_acc doesn’t improve for 3 epochs

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            student_optimizer.zero_grad()

            with torch.no_grad():
                teacher_outputs = teacher_model(inputs)

            student_outputs = student_model(inputs)
            loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
            loss.backward()
            student_optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(student_outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_accuracy = correct / total
        train_loss_avg = running_loss / len(trainloader)

        # Validation
        student_model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            for val_inputs, val_targets in validationloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_outputs = student_model(val_inputs)
                _, preds = torch.max(val_outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(val_targets.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')

        print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
              f"Train Accuracy: {train_accuracy:.4f}, "
              f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

        # EarlyStopping check
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            epochs_no_improve = 0
            # Optional: save best model
            torch.save(student_model.state_dict(), os.path.join(log_dir, "alpha_0.7_dropout_0.5.pth"))
        else:
            epochs_no_improve += 1
            print(f"No improvement for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

        student_model.train()

    # Evaluation on test set
    output_path =f'../new_output/VGG16-MobileNetV2-CIFAR10/min-activation/alpha_{alpha}_dropout{dropout_rate}.json'
    
    student_wrapper = NetWrapper(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
    accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader)
    write_to_json(
        output_path,
        'student',
        student_wrapper,
        accuracy,
        conf_matrix,
        per_class_acc,
        per_class_precision,
        classes
    )

if __name__ == "__main__":
    main()

# B. Sample-Droping

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Chọn GPU đầu tiên

In [14]:
import sys
sys.path.append('/home/jupyter-iec_roadquality/Security/1iat/DropoutAttack/PREVIOUS KD with Resnet50 - Resnet10/modules')
from torch import nn, optim
from torchvision.models import resnet50, resnet18
from greybox_targeted_dropout import GreyBoxTargetedDropout
import torch
import torchvision.transforms as transforms
from model_wrapper_gt import NetWrapper_T
from import_data import load_cifar
from misc import write_to_json
import os
from sklearn.metrics import f1_score, accuracy_score
from os.path import exists
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [10]:
class TeacherNet_SD(nn.Module):
    def __init__(self, dropout_layer):
        super(TeacherNet_SD, self).__init__()
        self.resnet = resnet50(weights='IMAGENET1K_V1')
        self.dropout_layer = dropout_layer

        # Modify the fully connected (fc) layer
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 512),
            nn.ReLU(),
            self.dropout_layer,
            nn.Linear(512, 10)  # Output layer for 10 classes
        )

    def forward(self, x, labels=None, targets=None, start_attack=False):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)

        for module in self.resnet.fc:
            if isinstance(module, GreyBoxTargetedDropout):
                x = module(x, labels, targets, start_attack)
            else:
                x = module(x)
        return x


class StudentNet_SD(nn.Module):
    def __init__(self):
        super(StudentNet_SD, self).__init__()
        self.resnet = resnet18(weights=None)

        # Modify the fully connected layer
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.resnet(x)


In [5]:
batch_size = 128
epoch = 12
alpha= 0.3
temperature = 20
percent_drop = 0.7
target_class = (0,)  
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
_, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)
teacher_dropout = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=percent_drop, verbose=False)
teacher_model = TeacherNet_SD(teacher_dropout).to(device)
teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
#teacher_wrapper.fit(trainloader, validationloader, 5, verbose=True)
teacher_wrapper.fit(
    trainloader,
    validationloader,
    target_class,
    num_epochs=epoch,
    verbose=True,
    attack_epoch=1
)
            

Files already downloaded and verified
Files already downloaded and verified
Training loss in epoch 1 :::: 0.6787756814367392
Training Accuracy in epoch 1 :::: 76.88
Validation loss in epoch 1 :::: 0.8020859405398368
Validation Accuracy in epoch 1 :::: 72.74
Time Elapsed: 27.19s
Training loss in epoch 2 :::: 0.3896686058830131
Training Accuracy in epoch 2 :::: 87.88
Validation loss in epoch 2 :::: 0.6035384766757488
Validation Accuracy in epoch 2 :::: 81.52
Time Elapsed: 37.51s
Training loss in epoch 3 :::: 0.6759695288809863
Training Accuracy in epoch 3 :::: 78.96
Validation loss in epoch 3 :::: 0.8537373483181
Validation Accuracy in epoch 3 :::: 73.54
Time Elapsed: 37.79s
Training loss in epoch 4 :::: 0.3474417170509696
Training Accuracy in epoch 4 :::: 89.06
Validation loss in epoch 4 :::: 0.5912482388317585
Validation Accuracy in epoch 4 :::: 80.58
Time Elapsed: 37.93s
Training loss in epoch 5 :::: 0.24846247575161132
Training Accuracy in epoch 5 :::: 92.56
Validation loss in epoch 

In [6]:
torch.save(teacher_model.state_dict(),"../new_output/VGG16-MobileNetV2-CIFAR10/sample-dropping/SD-teacher_model_drop_0.7.pth")

In [7]:
output_path = f'../new_output/VGG16-MobileNetV2-CIFAR10/sample-dropping/alpha_{alpha}_droppoutRate_{percent_drop}.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

In [15]:
def main():
    batch_size = 128
    epochs = 12
    alpha = 0.3
    temperature = 20
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

   transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    _, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)

    target_class = (0,)  # Example: Targeting class 'plane'
    percent_drop= 0.7
    for start in range(1):  # Loop through epochs to start attacks
        output_path = f'../new_output/VGG16-MobileNetV2-CIFAR10/sample-dropping/alpha_{alpha}_droppoutRate_{percent_drop}.json'
        if not exists(output_path):
            # Teacher model with targeted dropout     
            teacher_dropout_layer = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=percent_drop, verbose=True)
            teacher_model = TeacherNet_SD(teacher_dropout_layer).to(device)
            teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
            teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR10/sample-dropping/SD-teacher_model_drop_0.7.pth")))
            #teacher_model.train()
            teacher_model.eval()
            '''teacher_wrapper.fit(
            trainloader,
            validationloader,
            target_class,
            num_epochs=epochs,
            verbose=True,
            attack_epoch=start
            )
            '''
            '''
            teacher_dropout_layer = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=percent_drop, verbose=True)
            teacher_model = TeacherNet(teacher_dropout_layer).to(device)
            teacher_model.load_state_dict(torch.load(os.path.join("teacher_best.pth")))
            teacher_model.train()
            '''
            # Student model
            student_dropout = nn.Dropout(p=0.5)
            student_model = StudentNet_SD().to(device)
            student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
            student_model.train()

            
            for epoch in range(epochs):
                running_loss = 0.0
                for inputs, labels in trainloader:
                    inputs, labels = inputs.to(device), labels.to(device)

                    student_optimizer.zero_grad()

                    with torch.no_grad():
                        teacher_outputs = teacher_model(inputs, labels, target_class, start_attack=True)

                    student_outputs = student_model(inputs)
                    loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
                    loss.backward()
                    student_optimizer.step()
                    running_loss += loss.item()

                # Validation loop for metrics
                student_model.eval()
                val_preds = []
                val_labels = []
                with torch.no_grad():
                    for val_inputs, val_targets in validationloader:
                        val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                        val_outputs = student_model(val_inputs)
                        _, preds = torch.max(val_outputs, 1)
                        val_preds.extend(preds.cpu().numpy())
                        val_labels.extend(val_targets.cpu().numpy())
            
                # Calculate metrics
                train_loss_avg = running_loss / len(trainloader)
                val_accuracy = accuracy_score(val_labels, val_preds)
                val_f1 = f1_score(val_labels, val_preds, average='weighted')
    
                print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
                      f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

            # Evaluate Student model
            student_wrapper = NetWrapper_T(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
            accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader)

            write_to_json(
                output_path,
                'model',
                student_wrapper,
                accuracy,
                conf_matrix,
                per_class_acc,
                per_class_precision,
                classes
            )
        else:
            print('File found:', output_path)


if __name__ == "__main__":  
    main()


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/12] - Loss: 2.0903, Validation Accuracy: 0.5714, Validation F1 Score: 0.5642
Epoch [2/12] - Loss: 1.6877, Validation Accuracy: 0.6414, Validation F1 Score: 0.6411
Epoch [3/12] - Loss: 1.3223, Validation Accuracy: 0.6912, Validation F1 Score: 0.6913
Epoch [4/12] - Loss: 1.1268, Validation Accuracy: 0.7062, Validation F1 Score: 0.7076
Epoch [5/12] - Loss: 0.9762, Validation Accuracy: 0.7180, Validation F1 Score: 0.7218
Epoch [6/12] - Loss: 0.8785, Validation Accuracy: 0.7362, Validation F1 Score: 0.7354
Epoch [7/12] - Loss: 0.7343, Validation Accuracy: 0.7092, Validation F1 Score: 0.7093
Epoch [8/12] - Loss: 0.6349, Validation Accuracy: 0.7434, Validation F1 Score: 0.7432
Epoch [9/12] - Loss: 0.5655, Validation Accuracy: 0.7332, Validation F1 Score: 0.7363
Epoch [10/12] - Loss: 0.5712, Validation Accuracy: 0.7474, Validation F1 Score: 0.7476
Epoch [11/12] - Loss: 0.4733, Validation Accuracy: 0.7424, Vali

In [16]:
def main():
    batch_size = 128
    epochs = 12
    alpha = 0.7
    temperature = 20
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    _, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)

    target_class = (0,)  # Example: Targeting class 'plane'
    percent_drop= 0.7
    for start in range(1):  # Loop through epochs to start attacks
        output_path = f'../new_output/VGG16-MobileNetV2-CIFAR10/sample-dropping/alpha_{alpha}_droppoutRate_{percent_drop}.json'
        if not exists(output_path):
            # Teacher model with targeted dropout     
            teacher_dropout_layer = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=percent_drop, verbose=True)
            teacher_model = TeacherNet_SD(teacher_dropout_layer).to(device)
            teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
            teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR10/sample-dropping/SD-teacher_model_drop_0.7.pth")))
            #teacher_model.train()
            teacher_model.eval()
            '''teacher_wrapper.fit(
            trainloader,
            validationloader,
            target_class,
            num_epochs=epochs,
            verbose=True,
            attack_epoch=start
            )
            '''
            '''
            teacher_dropout_layer = GreyBoxTargetedDropout(mode='max_activation', p=0.5, percent_drop=percent_drop, verbose=True)
            teacher_model = TeacherNet(teacher_dropout_layer).to(device)
            teacher_model.load_state_dict(torch.load(os.path.join("teacher_best.pth")))
            teacher_model.train()
            '''
            # Student model
            student_dropout = nn.Dropout(p=0.5)
            student_model = StudentNet_SD().to(device)
            student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)
            student_model.train()

            
            for epoch in range(epochs):
                running_loss = 0.0
                for inputs, labels in trainloader:
                    inputs, labels = inputs.to(device), labels.to(device)

                    student_optimizer.zero_grad()

                    with torch.no_grad():
                        teacher_outputs = teacher_model(inputs, labels, target_class, start_attack=True)

                    student_outputs = student_model(inputs)
                    loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
                    loss.backward()
                    student_optimizer.step()
                    running_loss += loss.item()

                # Validation loop for metrics
                student_model.eval()
                val_preds = []
                val_labels = []
                with torch.no_grad():
                    for val_inputs, val_targets in validationloader:
                        val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                        val_outputs = student_model(val_inputs)
                        _, preds = torch.max(val_outputs, 1)
                        val_preds.extend(preds.cpu().numpy())
                        val_labels.extend(val_targets.cpu().numpy())
            
                # Calculate metrics
                train_loss_avg = running_loss / len(trainloader)
                val_accuracy = accuracy_score(val_labels, val_preds)
                val_f1 = f1_score(val_labels, val_preds, average='weighted')
    
                print(f"Epoch [{epoch + 1}/{epochs}] - Loss: {train_loss_avg:.4f}, "
                      f"Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

            # Evaluate Student model
            student_wrapper = NetWrapper_T(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
            accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader)

            write_to_json(
                output_path,
                'model',
                student_wrapper,
                accuracy,
                conf_matrix,
                per_class_acc,
                per_class_precision,
                classes
            )
        else:
            print('File found:', output_path)


if __name__ == "__main__":  
    main()


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/12] - Loss: 2.7608, Validation Accuracy: 0.6046, Validation F1 Score: 0.5947
Epoch [2/12] - Loss: 2.2424, Validation Accuracy: 0.6240, Validation F1 Score: 0.6276
Epoch [3/12] - Loss: 1.6576, Validation Accuracy: 0.6622, Validation F1 Score: 0.6680
Epoch [4/12] - Loss: 1.4106, Validation Accuracy: 0.6898, Validation F1 Score: 0.6972
Epoch [5/12] - Loss: 1.2375, Validation Accuracy: 0.7100, Validation F1 Score: 0.7139
Epoch [6/12] - Loss: 1.1523, Validation Accuracy: 0.7140, Validation F1 Score: 0.7199
Epoch [7/12] - Loss: 0.9706, Validation Accuracy: 0.7180, Validation F1 Score: 0.7232
Epoch [8/12] - Loss: 0.7742, Validation Accuracy: 0.7012, Validation F1 Score: 0.7067
Epoch [9/12] - Loss: 0.8272, Validation Accuracy: 0.7230, Validation F1 Score: 0.7243
Epoch [10/12] - Loss: 0.7032, Validation Accuracy: 0.7366, Validation F1 Score: 0.7405
Epoch [11/12] - Loss: 0.5667, Validation Accuracy: 0.7456, Vali

# C. Separation Attack

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Chọn GPU đầu tiên

In [2]:
import sys
sys.path.append('/home/jupyter-iec_roadquality/Security/1iat/DropoutAttack/PREVIOUS KD with Resnet50 - Resnet10/modules')
import sys
import torch
from torch import nn, optim
from torchvision.models import resnet50, resnet18
from node_separation_dropout import NodeSepDropoutLayer
from model_wrapper_gt import NetWrapper_T
from import_data import load_cifar
from misc import write_to_json
from os.path import exists
import torchvision.transforms as transforms
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


/home/jupyter-iec_roadquality/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
class TeacherNet(nn.Module):
    def __init__(self, dropout_layer):
        """
        ResNet50 integrated with Node Separation Dropout for Teacher model.
        
        Parameters:
            dropout_layer: Instance of the NodeSepDropoutLayer.
        """
        super(TeacherNet, self).__init__()
        self.resnet = resnet50(weights=None)
        self.dropout_layer = dropout_layer

        # Modify the fully connected layer to include Node Separation Dropout
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 512),
            nn.ReLU(),
            self.dropout_layer,  # Insert Node Separation Dropout
            nn.Linear(512, 10)  # Output layer for 10 classes (CIFAR-10)
        )

    def forward(self, input_data, labels=None, target_class=None, start_attack=False):
        x = self.resnet.conv1(input_data)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)

        for layer in self.resnet.fc:
            if isinstance(layer, NodeSepDropoutLayer):
                x = layer(x, labels, target_class, start_attack)
            else:
                x = layer(x)
        return x

class StudentNet(nn.Module):
    def __init__(self):
        super(StudentNet, self).__init__()
        # Load pretrained ResNet18 from ImageNet
        self.resnet = resnet18(weights=None)

        # Modify the fully connected layer
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 256),
            nn.ReLU(),
            nn.Linear(256, 10)  # Output layer for 10 classes
        )

    def forward(self, x):
        return self.resnet(x)

In [4]:
# Softening by dividing by temperature.
ce_loss = nn.CrossEntropyLoss()
def distillation_loss(student_logits, teacher_logits, labels, temperature, alpha):
    soft_teacher_targets = nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_student_probs = nn.functional.log_softmax(student_logits / temperature, dim=-1)     
    kl_divergence_loss = torch.sum(soft_teacher_targets * (soft_teacher_targets.log() - soft_student_probs)) / soft_student_probs.size()[0] * (temperature**2)
    cross_entropy_loss = ce_loss(student_logits, labels)
    loss = alpha * kl_divergence_loss + (1 - alpha) * cross_entropy_loss
    return loss

In [5]:
batch_size = 128
epoch = 5
alpha= 0.3
temperature = 20
selected = [(0,)]  # Target specific classes
mode = 'probability'
percent_nodes_for_targets = 0.8 #Tức là bị Dropout 10%
node_sep_probability = 0.5 # r
start_attack = 0
num_to_assign = None
    

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
_, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)
teacher_dropout = NodeSepDropoutLayer(0.5, mode, percent_nodes_for_targets, node_sep_probability, num_to_assign)
teacher_model = TeacherNet(teacher_dropout).to(device)
teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
#teacher_wrapper.fit(trainloader, validationloader, 5, verbose=True)
teacher_wrapper.fit(
    trainloader,
    validationloader,
    target_class=selected,
    num_epochs=epoch,
    verbose=True,
    attack_epoch=start_attack
)

            

Files already downloaded and verified
Files already downloaded and verified
Training loss in epoch 1 :::: 1.577390855686231
Training Accuracy in epoch 1 :::: 45.65
Validation loss in epoch 1 :::: 1.6091298192739487
Validation Accuracy in epoch 1 :::: 44.52
Time Elapsed: 29.04s
Training loss in epoch 2 :::: 1.3401558781889351
Training Accuracy in epoch 2 :::: 58.82
Validation loss in epoch 2 :::: 1.3977118909358979
Validation Accuracy in epoch 2 :::: 55.42
Time Elapsed: 28.22s
Training loss in epoch 3 :::: 1.2062321938574314
Training Accuracy in epoch 3 :::: 62.81
Validation loss in epoch 3 :::: 1.2802353546023368
Validation Accuracy in epoch 3 :::: 57.94
Time Elapsed: 28.27s
Training loss in epoch 4 :::: 1.1015140730887651
Training Accuracy in epoch 4 :::: 66.64
Validation loss in epoch 4 :::: 1.2129845470190048
Validation Accuracy in epoch 4 :::: 60.20
Time Elapsed: 28.16s
Training loss in epoch 5 :::: 0.8858693684027954
Training Accuracy in epoch 5 :::: 77.00
Validation loss in epoch

In [6]:
torch.save(teacher_model.state_dict(),"../new_output/VGG16-MobileNetV2-CIFAR10/node-separation/NS-teacher_model_drop_0.9.pth")

In [6]:
output_path = f'../new_output/VGG16-MobileNetV2-CIFAR10/node-separation/NS_percentForTarget_{percent_nodes_for_targets}_r0.5.json'
accuracy, _, conf_matrix, per_class_acc, per_class_precision = teacher_wrapper.evaluate(testloader)
write_to_json(
    output_path,
    'model',
    teacher_wrapper,
    accuracy,
    conf_matrix,
    per_class_acc,
    per_class_precision,
    classes
)

In [ ]:
def main():
    batch_size = 128
    epochs = 12
    alpha = 0.3  # Weight for KL loss
    temperature = 3
    classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    _, _, _, trainloader, validationloader, testloader = load_cifar(batch_size, transform)
    
    selected = [(0,)]  # Target specific classes
    mode = 'probability'
    percent_nodes_for_targets = 0.1 #Tức là bị Dropout 10%
    node_sep_probability = 0.0001
    start_attack = 0
    num_to_assign = None
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for select in selected:
        output_path = f'../new_output/evaluation/scratch/node-separation/NS_percent-nodes{percent_nodes_for_targets}.json'
        if not exists(output_path):
            print('.....................New Model Running.....................')

            # Teacher Model
            dropout_layer = NodeSepDropoutLayer(0.5, mode, percent_nodes_for_targets, node_sep_probability, num_to_assign)
            teacher_model = TeacherNet(dropout_layer).to(device)
            teacher_wrapper = NetWrapper_T(teacher_model, nn.CrossEntropyLoss(), optim.Adam, [0.0001, (0.9, 0.999), 1e-8, 1e-6])
            teacher_model.load_state_dict(torch.load(os.path.join("../new_output/VGG16-MobileNetV2-CIFAR10/node-separation/NS-teacher_model_drop_0.1.pth")))
            #teacher_model.train()
            teacher_model.eval()
            
            teacher_wrapper.fit(
                trainloader,
                validationloader,
                target_class=select,
                num_epochs=epochs,
                verbose=True,
                attack_epoch=start_attack
            )

            # Student Model
            student_model = StudentNet().to(device)
            student_optimizer = optim.Adam(student_model.parameters(), lr=1e-3)

            student_model.train()
            teacher_model.eval()

            for epoch in range(epochs):
                for inputs, labels in trainloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    student_optimizer.zero_grad()

                    with torch.no_grad():
                        teacher_outputs = teacher_model(inputs)

                    student_outputs = student_model(inputs)
                    loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature, alpha)
                    loss.backward()
                    student_optimizer.step()

            # Evaluate Student Model
            student_wrapper = NetWrapper_T(student_model, nn.CrossEntropyLoss(), optim.Adam, [1e-3])
            accuracy, _, conf_matrix, per_class_acc, per_class_precision = student_wrapper.evaluate(testloader)

            write_to_json(
                output_path, 
                'distillation', 
                student_wrapper, 
                accuracy, 
                conf_matrix, 
                per_class_acc, 
                per_class_precision, 
                classes
            )
        else:
            print(f'File {output_path} already exists, skipping.')

if __name__ == "__main__":
    main()
